<a href="https://colab.research.google.com/github/HajarGhizlaneML/machine_learning_project/blob/master/Evaluate_text.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

### Evaluate my text
Maintenant que le méthode de décodage est définie, nous pouvons écrire des fonctions pour évaluer une phrase d'entrée. **evaluatefonction** gère le processus de bas niveau de gestion de la phrase d'entrée. Nous formaterons d'abord la phrase comme un lot d'entrée d'index de mots avec batch_size == 1 . Pour cela, nous convertissons les mots de la phrase en leurs index correspondants et transposons les dimensions pour préparer le tenseur pour le modèle. Nous créons également un lengthstenseur qui contient la longueur de la phrase d'entrée. Dans ce cas, lengths est scalaire car nous n'évaluons qu'une seule phrase à la fois (batch_size == 1). Ensuite, nous obtenons le tenseur de phrase de réponse décodé en utilisant **GreedySearchDecoder** ( searcher). Enfin, nous convertissons les index de la réponse en mots et retournons la liste des mots décodés.

**evaluateInputagit** est comme une interface utilisateur pour le chatbot. Lorsqu'il est appelé, un champ de texte apparaîtra dans lequel nous pouvons saisir notre phrase de requête. Après avoir tapé la phrase d'entrée et appuyé sur Entrée , le texte est normalisé de la même manière que le training data, et transmis à la **evaluatefonction** pour obtenir une phrase de sortie décodée. Nous bouclons ce processus, afin que nous puissions continuer à discuter avec le bot jusqu'à ce que nous entrions «q» ou «quit».

Enfin, si un input qui contient un mot qui n'est pas dans le vocabulaire, nous gérons cela en envoyant un message d'erreur et en invitant l'utilisateur à entrer une autre phrase.

In [0]:
def evaluate(encoder, decoder, searcher, voc, sentence, max_length=MAX_LENGTH):
    ### Format input sentence as a batch
    # words -> indexes
    indexes_batch = [indexesFromSentence(voc, sentence)]
    # Create lengths tensor
    lengths = torch.tensor([len(indexes) for indexes in indexes_batch])
    # Transpose dimensions of batch to match models' expectations
    input_batch = torch.LongTensor(indexes_batch).transpose(0, 1)
    # Use appropriate device
    input_batch = input_batch.to(device)
    lengths = lengths.to(device)
    # Decode sentence with searcher
    tokens, scores = searcher(input_batch, lengths, max_length)
    # indexes -> words
    decoded_words = [voc.index2word[token.item()] for token in tokens]
    return decoded_words


def evaluateInput(encoder, decoder, searcher, voc):
    input_sentence = ''
    while(1):
        try:
            # Get input sentence
            input_sentence = input('> ')
            # Check if it is quit case
            if input_sentence == 'q' or input_sentence == 'quit': break
            # Normalize sentence
            input_sentence = normalizeString(input_sentence)
            # Evaluate sentence
            output_words = evaluate(encoder, decoder, searcher, voc, input_sentence)
            # Format and print response sentence
            output_words[:] = [x for x in output_words if not (x == 'EOS' or x == 'PAD')]
            print('Bot:', ' '.join(output_words))

        except KeyError:
            print("Error: Encountered unknown word.")